In [2]:
# ============================================================
# NOTEBOOK CONFIG — API KEY, Kaggle Secrets, Warnings
# (Do NOT put this inside JobPilot/agent.py)
# ============================================================

import os
import warnings
from kaggle_secrets import UserSecretsClient

warnings.filterwarnings("ignore")

# ============================================================
# API KEY SETUP (Notebook-only)
# ============================================================

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("🔑 Google API key successfully loaded from Kaggle Secrets.")
except Exception as e:
    print("❌ Failed to load Google API Key from Kaggle Secrets.")
    print("   Make sure you added 'GOOGLE_API_KEY' to Kaggle Secrets.")
    print("   Details:", e)


# ============================================================
# Logging set up
# ============================================================

import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)


print("✅ Global LLM client (GEMINI_CLIENT) is ready.")

print("✅ Notebook environment configured.")
print("   - API key available in environment")
print("   - Warnings disabled")
print("   - Logging configured")


🔑 Google API key successfully loaded from Kaggle Secrets.
✅ Global LLM client (GEMINI_CLIENT) is ready.
✅ Notebook environment configured.
   - API key available in environment
   - Warnings disabled
   - Logging configured


In [3]:
# ============================================================
# Config, Imports, Models, Session Service
# ============================================================

import os
import json
import hashlib
from typing import List, Dict, Any

from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.tools.agent_tool import AgentTool, ToolContext
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.google_search_tool import google_search


# ============================================================
# Retry config (shared)
# ============================================================
# ============================================================
# Core LLM Client (For Tools)
# ============================================================


retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Base model for most agents/tools
gemini_flash = Gemini(
    model="gemini-2.5-flash",
    retry_options=retry_config,
)

gemini_lite = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
)

# ============================================================
# Session Service (Persistent)
# ============================================================

session_service = DatabaseSessionService(db_url="sqlite:///autoapply_sessions.db")

print("succesful")

succesful


In [4]:
# ============================================================
# CELL 0 — GLOBAL SCHEMAS FOR JOBPILOT (Shared Across System)
# ============================================================

import json

# ------------------------------------------------------------
# PROFILE_SCHEMA
# ------------------------------------------------------------
PROFILE_SCHEMA = {
    "type": "object",
    "description": "A structured professional profile created by parsing a resume or user input.",
    "properties": {
        "name": {"type": "string", "description": "The full name of the user."},
        "location": {"type": "string", "description": "The user's current city and state/country (e.g., 'San Francisco, CA')."},
        "contact": {
            "type": "object",
            "properties": {
                "email": {"type": "string", "description": "Primary email address."},
                "phone": {"type": "string", "description": "Primary phone number."},
                "linkedin_url": {"type": "string", "description": "URL of the user's professional profile (LinkedIn/Portfolio)."}
            }
        },
        "education": {
            "type": "array",
            "description": "A list of academic degrees or certifications.",
            "items": {
                "type": "object",
                "properties": {
                    "institution": {"type": "string"},
                    "degree": {"type": "string"},
                    "focus": {"type": "string", "description": "Major or area of study."},
                    "dates": {"type": "string", "description": "Graduation year or date range (e.g., '2015-2019')."}
                }
            }
        },
        "experience": {
            "type": "array",
            "description": "A list of work experiences.",
            "items": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "company": {"type": "string"},
                    "dates": {"type": "string", "description": "Date range (e.g., 'Jan 2020 - Present')."},
                    "description": {"type": "string", "description": "A summary or key bullet points of responsibilities/achievements."}
                }
            }
        },
        "skills": {
            "type": "array",
            "description": "A list of technical and soft skills.",
            "items": {"type": "string"}
        },
        "job_preferences": {
            "type": "object",
            "description": "The user's desired job criteria.",
            "properties": {
                "role_types": {"type": "array", "items": {"type": "string"}},
                "industries": {"type": "array", "items": {"type": "string"}},
                "locations": {"type": "array", "items": {"type": "string"}},
                "remote": {"type": ["boolean", "null"]},
                "salary_min": {"type": ["integer", "null"]},
                "number_of_jobs_wanted": {
                    "type": "integer",
                    "description": "How many job matches the user asked for."
                }
            }
        },
        "additional_notes": {"type": "string", "description": "Any remaining unparsed or miscellaneous but important text."}
    },
    "required": ["name", "skills"]
}

# Used in:
#   - profile_builder_tool (input + output)
#   - orchestrator_agent (input routing + validation)
#   - job_search_agent (input)
#   - resume_generator_agent (input)
#   - cover_letter_agent (input)
#   - application_builder_agent (input)


# ------------------------------------------------------------
# JOB_FINDER_OUTPUT_SCHEMA
# ------------------------------------------------------------
JOB_FINDER_OUTPUT_SCHEMA = {
    "type": "object",
    "description": "A set of job posting URLs produced by searching for roles that fit the user's profile.",
    "properties": {
        "query_used": {
            "type": "string",
            "description": "The exact search query used for the Google Custom Search API."
        },
        "job_links": {
            "type": "array",
            "description": "A list of job posting URLs.",
            "items": {"type": "string"}
        },
        "num_results_requested": {
            "type": "integer",
            "description": "The number of results the caller requested."
        },
        "num_results_returned": {
            "type": "integer",
            "description": "The number of valid job posting URLs actually returned."
        },
        "error": {
            "type": ["string", "null"],
            "description": "Error details if the search failed; null if successful."
        }
    },
    "required": ["query_used", "job_links", "num_results_requested", "num_results_returned"]
}

# Used in:
#   - job_finder_tool
#   - job_search_agent



# Used in:
#   - fetch_job_tool
#   - job_search_agent


# ------------------------------------------------------------
# JOB_DETAILS_SCHEMA
# ------------------------------------------------------------
JOB_DETAILS_SCHEMA = {
    "type": "object",
    "description": "A structured representation of a job posting extracted from raw HTML/text.",
    "properties": {
        "job_id": {
            "type": "string",
            "description": "A deterministic hashed ID of the job posting URL."
        },
        "title": {
            "type": "string",
            "description": "The job title extracted from the posting (e.g. 'Data Scientist')."
        },
        "company": {
            "type": "string",
            "description": "The hiring company's name."
        },
        "location": {
            "type": "string",
            "description": "The job's listed location (city/state/country)."
        },
        "employment_type": {
            "type": "string",
            "description": "Full-Time, Part-Time, Internship, Contract, etc."
        },
        "salary": {
            "type": ["string", "null"],
            "description": "Salary range if specified in the posting."
        },
        "job_description": {
            "type": "string",
            "description": "Full descriptive text of the job's responsibilities."
        },
        "requirements": {
            "type": "array",
            "description": "Bullet points or extracted skill requirements.",
            "items": {"type": "string"}
        },
        "qualifications": {
            "type": "array",
            "description": "Education/experience qualification bullet points.",
            "items": {"type": "string"}
        },
        "skills_mentioned": {
            "type": "array",
            "description": "Extracted list of skills explicitly mentioned in the posting.",
            "items": {"type": "string"}
        },
        "apply_url": {
            "type": "string",
            "description": "The canonical application URL for the job posting."
        }
    },
    "required": ["job_id", "title", "company", "job_description", "apply_url"]
}


# Used in:
#   - extract_job_details_tool (output)
#   - job_filter_agent (input)
#   - job_search_agent (loop processing)
#   - job_summarizer_agent (input)
#   - resume_generator_agent (input)
#   - cover_letter_agent (input)
#   - application_builder_agent (selected job input)

FETCH_JOB_SCHEMA = {
    "type": "object",
    "description": "The raw content of a job posting fetched from the web.",
    "properties": {
        "url": {
            "type": "string",
            "description": "The job posting URL that was fetched."
        },
        "status": {
            "type": "integer",
            "description": "HTTP status code returned by the server."
        },
        "success": {
            "type": "boolean",
            "description": "Whether the job page was successfully fetched."
        },
        "content": {
            "type": "string",
            "description": "Raw HTML or plaintext content of the job page. Empty string if fetch failed."
        },
        "error": {
            "type": ["string", "null"],
            "description": "Error details if the fetch was unsuccessful."
        }
    },
    "required": ["url", "success", "status", "content"]
}



# ------------------------------------------------------------
# JOB_SUMMARY_SCHEMA
# ------------------------------------------------------------
JOB_SUMMARY_SCHEMA = {
    "type": "object",
    "description": "Summary of a job posting produced by job_summarizer_agent.",
    "properties": {
        "job_id": {"type": "string"},
        "summary": {"type": "string"},
        "score": {"type": "integer"},
        "link": {"type": "string"}
    },
    "required": ["job_id", "summary", "score", "link"]
}

# Used in:
#   - job_summarizer_agent


# ------------------------------------------------------------
# RANK_JOB_SCHEMA
# ------------------------------------------------------------
RANK_JOB_SCHEMA = {
    "type": "object",
    "description": "Ranking output from rank_job_tool (Agent 1 final stage).",
    "properties": {
        "ranked_jobs": {"type": "array", "items": {"type": "object"}},
        "top_k": {"type": "integer"},
        "total_jobs_in": {"type": "integer"},
        "total_jobs_ranked": {"type": "integer"}
    },
    "required": ["ranked_jobs", "top_k", "total_jobs_in", "total_jobs_ranked"]
}


JOB_FILTER_OUTPUT_SCHEMA = {
    "type": "object",
    "properties": {
        "pass": {"type": "boolean"},
        "score": {"type": "integer"},
        "rationale": {"type": "string"}
    },
    "required": ["pass", "score", "rationale"]
}


# Used in:
#   - rank_job_tool (output)
#   - job_search_agent (final output to orchestrator)


print("YAYY")

YAYY


In [ ]:
schemas = [PROFILE_SCHEMA, JOB_FINDER_OUTPUT_SCHEMA, JOB_DETAILS_SCHEMA, FETCH_JOB_SCHEMA, JOB_SUMMARY_SCHEMA, RANK_JOB_SCHEMA,
           JOB_FILTER_OUTPUT_SCHEMA]

In [5]:
# ============================================================
# CELL — Normalize Job Details Tool (Deterministic)
# ============================================================

import hashlib
from typing import Dict, Any
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.agent_tool import ToolContext


def normalize_job_details(tool_context: ToolContext, url: str, agent_output: Dict[str, Any]) -> Dict[str, Any]:
    """
    Deterministic post-processing for job details extracted by extract_job_details_agent.

    Responsibilities:
    - Compute deterministic job_id = sha256(url)[:16]
    - Always set apply_url = url
    - Ensure all fields required by JOB_DETAILS_SCHEMA exist
    - Fill missing values with correct defaults:
        - "" for string fields
        - [] for array fields
        - None for nullables
    - NEVER modify or infer semantic content
    """

    # ---------------------------------------------------------
    # 1. Compute deterministic job_id
    # ---------------------------------------------------------
    job_id = hashlib.sha256(url.encode()).hexdigest()[:16]

    # Start with the agent output (may have missing fields)
    structured = dict(agent_output)

    # ---------------------------------------------------------
    # 2. Force deterministic fields
    # ---------------------------------------------------------
    structured["job_id"] = job_id
    structured["apply_url"] = url

    # ---------------------------------------------------------
    # 3. Enforce schema completeness
    # ---------------------------------------------------------
    for key, spec in JOB_DETAILS_SCHEMA["properties"].items():
        if key not in structured:
            # Determine correct default by schema type
            t = spec["type"]

            if t == "string":
                structured[key] = ""

            elif t == "array":
                structured[key] = []

            elif t == "number":
                structured[key] = None

            elif t == "integer":
                structured[key] = None

            elif t == "boolean":
                structured[key] = None

            elif t == "null":
                structured[key] = None

            elif isinstance(t, list):
                # e.g. ["string", "null"]
                if "string" in t:
                    structured[key] = ""
                else:
                    structured[key] = None

            else:
                # Fallback safe default
                structured[key] = None

    return structured


# ---------------------------------------------------------
# ADK Registration
# ---------------------------------------------------------
normalize_job_details_tool = FunctionTool(
    func=normalize_job_details
)

print("✅ normalize_job_details_tool registered (deterministic, schema-safe).")


✅ normalize_job_details_tool registered (deterministic, schema-safe).


tools

In [21]:
# ============================================================
# CELL — Job Finder Tool (High-Accuracy Version)
# ============================================================
import json
from typing import Dict, Any, List
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.agent_tool import ToolContext
from google.adk.tools.google_search_tool import google_search


# ------------------------------------------------------------
# FUNCTION TOOL — Job Finder
# ------------------------------------------------------------
def job_finder_tool(tool_context: ToolContext, profile: Dict[str, Any], num_results: int = 10) -> Dict[str, Any]:
    """
    Build a job search query from the user's profile and return a list of job posting URLs.

    This tool **must remain deterministic**:
    - NO LLM calls
    - NO reasoning steps
    - Only uses the ADK google_search tool
    - MUST return structured JSON conforming to JOB_FINDER_OUTPUT_SCHEMA

    INPUT:
    {
        "profile": <object following ROFILE_SCHEMA>,          # user profile (from profile_builder_tool)
        "num_results": <int>       
    }

    OUTPUT (STRICT):
    <object following JOB_FINDER_OUTPUT_SCHEMA>
    """

    # -------------------------------
    # Extract job preferences
    # -------------------------------
    session = tool_context.state


    prefs = profile.get("job_preferences", {})

    role_types = prefs.get("role_types", [])
    industries = prefs.get("industries", [])
    locations = prefs.get("locations", [])
    remote_pref = prefs.get("remote", None)

    # -------------------------------
    # Build dynamic search query
    # -------------------------------
    # Role types (e.g., "Data Scientist", "Machine Learning Engineer")
    role_part = " ".join(role_types) if role_types else ""

    # Industries (e.g., "AI", "FinTech")
    industry_part = " ".join(industries) if industries else ""

    # Locations
    location_part = " ".join(locations) if locations else ""

    # Remote keyword
    remote_part = "remote" if remote_pref else ""

    # Build final query (clean and compact)
    components = [role_part, industry_part, remote_part, location_part, "jobs"]
    query = " ".join([c for c in components if c.strip() != ""]).strip()

    # Use fallback if the profile has minimal info
    if query == "":
        query = "entry level jobs remote"

    # -------------------------------
    # Perform search with ADK google_search tool
    # -------------------------------
    try:
        search_results = google_search(
            query=query,
            num_results=num_results
        )
    except Exception as e:
        # Return structured failure output
        return {
            "query_used": query,
            "job_links": [],
            "num_results_requested": num_results,
            "num_results_returned": 0,
            "error": f"SEARCH_EXCEPTION: {str(e)}"
        }

    # -------------------------------
    # Extract links only
    # -------------------------------
    job_links: List[str] = []
    for item in search_results:
        link = item.get("link")
        if isinstance(link, str) and link.startswith(("http://", "https://")):
            job_links.append(link)

    # -------------------------------
    # Structured output
    # -------------------------------
    return {
        "query_used": query,
        "job_links": job_links,
        "num_results_requested": num_results,
        "num_results_returned": len(job_links),
        "error": None
    }


# ------------------------------------------------------------
# ADK WRAPPER
# ------------------------------------------------------------
job_finder_tool_adk = FunctionTool(
    func=job_finder_tool,
)

print("✅ Job Finder Tool created with full schema and deterministic behavior.")


✅ Job Finder Tool created with full schema and deterministic behavior.


In [7]:
# ============================================================
# CELL — Fetch Job Tool
# ============================================================

import requests
import json
from typing import Dict, Any
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.agent_tool import ToolContext


# ------------------------------------------------------------
# FUNCTION TOOL — Fetch Job Page
# ------------------------------------------------------------
def fetch_job_tool(tool_context: ToolContext, url: str) -> Dict[str, Any]:
    """
    Fetch the raw HTML/text content of a job posting from a URL.

    This tool performs a *deterministic* HTTP GET request — meaning:
    - NO LLM calls are used
    - NO reasoning
    - Must return only structured JSON (FETCH_JOB_SCHEMA)

    INPUT:
    {
        "url": "<job posting url>"
    }

    OUTPUT (STRICT):
    <object following FETCH_JOB_SCHEMA>
    """

    session = tool_context.state

    # Validate input
    if not isinstance(url, str) or not url.startswith(("http://", "https://")):
        return {
            "url": url,
            "status": 0,
            "success": False,
            "content": "",
            "error": "Invalid URL format."
        }

    try:
        # Use requests to perform deterministic fetch
        response = requests.get(url, timeout=10)

        # Successful HTTP response
        if response.status_code == 200:
            return {
                "url": url,
                "status": response.status_code,
                "success": True,
                "content": response.text[:500000],  # safety limit: prevent enormous pages
                "error": None,
            }

        # Non-200 status codes still return structured JSON
        else:
            return {
                "url": url,
                "status": response.status_code,
                "success": False,
                "content": "",
                "error": f"HTTP error {response.status_code}",
            }

    except Exception as e:
        # Full failure: network issue, timeout, DNS, etc.
        return {
            "url": url,
            "status": 0,
            "success": False,
            "content": "",
            "error": f"REQUEST_EXCEPTION: {str(e)}"
        }


# ------------------------------------------------------------
# ADK WRAPPER
# ------------------------------------------------------------
fetch_job_tool_adk = FunctionTool(
    func=fetch_job_tool,
)

print("✅ Fetch Job Tool created with schema enforcement.")


✅ Fetch Job Tool created with schema enforcement.


In [8]:
# ============================================================
# CELL — Rank Job Tool (Deterministic, Schema-Aligned)
# ============================================================

from typing import Dict, Any, List
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.agent_tool import ToolContext


def rank_job_tool(tool_context: ToolContext, jobs: List[Dict[str, Any]], top_k: int) -> Dict[str, Any]:
    """
    Deterministically rank jobs by score (descending) and return top_k.

    This tool performs NO LLM calls, NO reasoning.
    It only:
        - sorts the jobs list by job["score"]
        - selects the top_k entries
        - returns structured JSON conforming to RANK_JOB_SCHEMA

    INPUT:
    {
        "jobs": [
            {
                "job_id": <string>,
                "title": <string>,
                "company": <string>,
                ...
                "score": <0–100>,     # REQUIRED for ranking
                "pass": <bool>        # from job_filter_agent
            },
            ...
        ],
        "top_k": <int>
    }

    OUTPUT (STRICT):
    {
        "ranked_jobs": [... top_k job dicts ...],
        "top_k": <int>,
        "total_jobs_in": <int>,
        "total_jobs_ranked": <int>
    }
    """

    session = tool_context.state


    # -------------------------------
    # Input validation
    # -------------------------------
    if not isinstance(jobs, list):
        return {
            "ranked_jobs": [],
            "top_k": top_k,
            "total_jobs_in": 0,
            "total_jobs_ranked": 0,
            "error": "Invalid input: jobs must be a list."
        }

    # Filter out jobs missing scores (safety)
    valid_jobs = [j for j in jobs if isinstance(j.get("score"), (int, float))]

    # -------------------------------
    # Deterministic sort by score
    # -------------------------------
    ranked = sorted(valid_jobs, key=lambda j: j["score"], reverse=True)

    # Select top_k
    top_ranked = ranked[:top_k]

    # -------------------------------
    # Structured output
    # -------------------------------
    return {
        "ranked_jobs": top_ranked,
        "top_k": top_k,
        "total_jobs_in": len(jobs),
        "total_jobs_ranked": len(top_ranked)
    }


# ------------------------------------------------------------
# ADK WRAPPER
# ------------------------------------------------------------
rank_job_tool_adk = FunctionTool(
    func=rank_job_tool,
)

print("✅ Rank Job Tool created (deterministic, schema-aligned).")


✅ Rank Job Tool created (deterministic, schema-aligned).


In [ ]:
schemas = [PROFILE_SCHEMA, JOB_FINDER_OUTPUT_SCHEMA, JOB_DETAILS_SCHEMA, FETCH_JOB_SCHEMA, JOB_SUMMARY_SCHEMA, RANK_JOB_SCHEMA,
           JOB_FILTER_OUTPUT_SCHEMA]

In [26]:
# ============================================================
# CELL — Orchestrator Agent (High-Accuracy, Tool-Consistent, Updated Schemas)
# ============================================================

from google.adk.agents import LlmAgent


orchestrator_agent = LlmAgent(
    model=gemini_flash,
    name="orchestrator_agent",
    description=(
        "Top-level controller for the JobPilot multi-agent system. "
        "Handles profile parsing, job search, summarization, "
        "and job application document generation."
    ),
    instruction="""
You are the **Orchestrator Agent** for JobPilot — the top-level coordinator of a multi-agent job search and application-building system.

You NEVER perform the work yourself.  
You ALWAYS call tools and other agents.

Follow EXACTLY this workflow:

==============================================================
1. USER INPUT → PROFILE (via profile_builder_tool)
==============================================================

You ALWAYS start with a raw user message containing free-form professional background.

    let user_text = <the EXACT raw user message>

Then call:

    profile_builder_agent_adk:
        Input:
            {
                "user_text": user_text
            }

The tool returns:

{
  "profile": <object following PROFILE_SCHEMA>
      
}

You MUST store this as the user's profile. 


==============================================================
2. TRIGGER JOB SEARCH AGENT
==============================================================

Call **job_search_agent** with:

{
    "profile": <object following PROFILE_SCHEMA>,
    "rejection_memory": <list or dict from long-term memory (may be empty)>
}

job_search_agent will:

- Use job_finder_tool → returns:
    <JOB_FINDER_OUTPUT_SCHEMA>

- Use fetch_job_tool → returns:
    <FETCH_JOB_SCHEMA>

- Use extract_job_details_agent → returns:
    <JOB_DETAILS_SCHEMA>

- Use job_filter_agent → returns:
    <JOB_FILTER_OUTPUT_SCHEMA>

- Use rank_job_tool → returns:
    {
        "ranked_jobs": [<JOB_DETAILS_SCHEMA> + <JOB_FILTER_OUTPUT_SCHEMA>],
        "top_k": <int>,
        "total_jobs_in": <int>,
        "total_jobs_ranked": <int>
    }

job_search_agent returns:
{
    "ranked_jobs": [<JOB_DETAILS_SCHEMA> + <JOB_FILTER_OUTPUT_SCHEMA>]
}



==============================================================
3. JOB SUMMARIZATION
==============================================================

For each job in ranked_jobs:

Call job_summarizer_agent with:
{
    "job": <JOB_DETAILS_SCHEMA + JOB_FILTER_OUTPUT_SCHEMA>
}

Expect response:
{
    "job_id": <string>,
    "summary": <string>,
    "score": <int>,
    "link": <string>
}


==============================================================
4. HANDLE USER FEEDBACK
==============================================================

From user reply, extract:

- selected_jobs: the ones user wants to apply to
- rejection_reasons: reasons for rejecting the others

Store rejection_reasons into long-term memory under key "rejection_memory".


==============================================================
5. TRIGGER APPLICATION BUILDER AGENT
==============================================================

Call application_builder_agent with:

{
    "selected_jobs": [...],
    "user_profile": <profile>
}

It returns:
{
    "applications": [
        {
            "job_id": <string>,
            "resume_text": <string>,
            "cover_letter_text": <string>
        },
        ...
    ]
}


==============================================================
6. RETURN FINAL OUTPUT
==============================================================

You MUST return all generated application documents to the user.


==============================================================
RULES
==============================================================

- ALWAYS call profile_builder_tool first using user_text.
- NEVER modify the profile manually — only the tool may update it.
- NEVER create job details manually — rely on extraction tool.
- NEVER generate resumes or cover letters — use application_builder_agent.
- Long-term memory:
    - "user_profile"
    - "rejection_memory"
- Session memory:
    - Temporary job lists, search results, intermediate data.

Your role is sequencing and routing — not reasoning or generating content yourself.

-When calling tools or agents:

 - NEVER wrap inputs inside {"request": ... }
 - ALWAYS send arguments as a JSON object exactly matching the expected signature.

Example:
Call job_search_agent with:
{
    "profile": <PROFILE_SCHEMA object>,
    "rejection_memory": <list>
}

"""
)

print("✅ Orchestrator Agent updated with complete schemas.")


✅ Orchestrator Agent updated with complete schemas.


In [46]:
class ProfileBuilderInput(BaseModel):
    user_text: str



# ============================================================
# CELL — Profile Builder Agent (LLM-based, replaces old tool)
# ============================================================

from google.adk.agents import LlmAgent

profile_builder_agent = LlmAgent(
    model=gemini_flash,
    name="profile_builder_agent",
    description=(
        "Parses the user's free-form background into PROFILE_SCHEMA. "
        "This replaces the deterministic profile_builder_tool."
    ),
    input_schema=ProfileBuilderInput,
    static_instruction="""
You are the **Profile Builder Agent** for JobPilot.

Your job is to:
1. Read the user's raw free-form text (resume-like content).
2. Decide whether this is a NEW profile or an UPDATE.
3. If "user_profile" already exists in long-term memory AND the new text does not explicitly indicate an update, simply return the existing profile.
4. Otherwise, rebuild the entire profile from scratch using the LLM.
5. Output MUST match **PROFILE_SCHEMA exactly**.

==============================================================
INPUT FORMAT (from Orchestrator)
==============================================================

You will ALWAYS receive:

{
    "user_text": "<raw free-form text>",
    "existing_profile": <object or null>,   # pulled from memory by Orchestrator
}

==============================================================
DETECTING USER INTENT TO UPDATE
==============================================================

The user is considered to be updating their profile if the message contains ANY of:

"update", "change", "modify", "add new info",
"correct my profile", "here is new info",
"updated details", "resume", "new details"

If NONE of these appear AND existing_profile is NOT null:
    → RETURN:
    {
        "profile": existing_profile
    }

==============================================================
WHEN BUILDING A NEW PROFILE
==============================================================

1. Read the user_text carefully.
2. Extract fields strictly according to PROFILE_SCHEMA.
3. Missing information MUST be empty strings, empty lists, or null — NEVER invent details.
4. Add these two non-schema fields after extraction:
    "update_required": false
    "last_update": <UNIX timestamp>

==============================================================
PROFILE_SCHEMA
==============================================================

Return a JSON object matching EXACTLY the following schema:

{json.dumps(PROFILE_SCHEMA, indent=2)}

==============================================================
OUTPUT FORMAT
==============================================================

You MUST output:

{{
    "profile": <PROFILE_SCHEMA object>
}}

This is the ONLY valid output. No explanation text.

==============================================================
RULES
==============================================================

- NEVER invent facts — missing info must be empty.
- NEVER embed commentary or system notes inside profile fields.
- NEVER return text outside the JSON.
- Output must be valid JSON that conforms exactly to PROFILE_SCHEMA.
"""
)

print("✅ Profile Builder Agent created.")


✅ Profile Builder Agent created.


In [42]:
# ============================================================
# CELL — Job Search Agent (Agent 1) + Sub-Agent Schema-Aligned
# ============================================================

from google.adk.agents import LlmAgent


# ------------------------------------------------------------
# JOB FILTER AGENT (Sub-Agent)
# ------------------------------------------------------------
job_filter_agent = LlmAgent(
    model=gemini_lite,
    name="job_filter_agent",
    description="Evaluates user–job fit and produces a binary pass/fail and a numeric score based on profile and rejection memory.",
    static_instruction="""
You are the **Job Filter Agent** in JobPilot.

Your job:
Given:
    - job_details: a structured job posting JSON (from extract_job_details_tool)
    - profile: the user's structured profile
    - rejection_memory: a long-term memory structure describing past user dislikes

EXPECTED INPUT
--------------
{
    "job_details": <object following JOB_DETAILS_SCHEMA>,
    "profile": <object following PROFILE_SCHEMA>,
    "rejection_memory": <list or object>
}

OUTPUT
------

{
    "pass": <true/false>,
    "score": <0–100>,
    "rationale": "<short explanation>"
}

RULES
-----

1. Score range:
   - If job is strongly mismatched → <40
   - If partially matched → 40–69
   - If well matched → 70+
   - Required skills missing → subtract
   - Conflicts with rejection_memory → subtract significantly

2. Binary:
   - pass = (score >= 60) unless user preferences explicitly oppose the job.
   - The score and pass MUST be logically consistent.

3. NEVER fabricate missing job info.

4. Output MUST be strictly valid JSON.
"""
)

# ============================================================
# CELL — Extract Job Details Agent (Optimized, ADK-correct)
# ============================================================

from google.adk.agents import LlmAgent
import json

extract_job_details_agent = LlmAgent(
    model=gemini_flash,
    name="extract_job_details_agent",
    description=(
        "Extracts semantic job information from raw HTML/text. "
        "Does NOT handle deterministic cleanup; that is done by normalize_job_details_tool."
    ),
    static_instruction="""
You are the **Extract Job Details Agent** for JobPilot.

Your role:
Given a job posting URL and its raw HTML/text content, extract ONLY the semantic fields
that are present in the job posting. You MUST NOT perform normalization or add deterministic
fields like job_id or apply_url. A separate tool will handle that afterward.

==============================================================
EXPECTED INPUT
==============================================================

{
    "url": "<string>",
    "raw_content": "<string>"
}

==============================================================
FIELDS YOU MAY EXTRACT (subset of JOB_DETAILS_SCHEMA)
==============================================================

{json.dumps({k: v for k, v in JOB_DETAILS_SCHEMA['properties'].items()
             if k not in ['job_id', 'apply_url']}, indent=2)}

These are the ONLY keys you may return.

==============================================================
RULES FOR EXTRACTION
==============================================================

1. NEVER fabricate information.
   - If a field is not found, OMIT it entirely.
   - DO NOT add empty strings, empty lists, or nulls — that is handled elsewhere.

2. DO NOT include these fields:
   - "job_id"
   - "apply_url"

3. DO NOT perform deterministic adjustments.
   Your job is **semantic extraction only**.

4. Output only the fields you find, in valid JSON.
   No explanations, no surrounding text.

==============================================================
OUTPUT FORMAT
==============================================================

Return ONLY a JSON object containing the keys you successfully extracted.

Example (if only title and company were found):

{
    "title": "Data Analyst",
    "company": "Acme Corp"
}

This is the ONLY valid type of output.
"""
)

print("✅ Extract Job Details Agent updated (semantic only, ADK-correct).")


from pydantic import BaseModel, Field
from typing import List, Dict, Any
# from adk import LlmAgent, gemini_flash # (Add other imports you need)

# --- START of the Pydantic Class ---
class JobSearchAgentInput(BaseModel):
    """
    Defines the expected input structure for the JobSearchAgent.
    This replaces the JSON Schema dictionary.
    """
    profile: Dict[str, Any] = Field(
        ..., 
        description="The user's complete profile object, including job preferences."
    )
    rejection_memory: List[Any] = Field(
        ...,
        description="A list of jobs or IDs that have already been rejected."
    )
# --- END of the Pydantic Class ---

# ------------------------------------------------------------
# JOB SEARCH AGENT (Agent 1)
# ------------------------------------------------------------
job_search_agent = LlmAgent(
    model=gemini_flash,
    name="job_search_agent",
    description=(
        "Agent 1 in JobPilot. Performs the full job-search loop using "
        "job_finder_tool, fetch_job_tool, extract_job_details_tool, "
        "job_filter_agent, and rank_job_tool."
    ),
    input_schema=JobSearchAgentInput,
    
    instruction="""
You are **Agent 1 — Job Search Agent** in JobPilot.

Your job is to run a COMPLETE JOB-SEARCH LOOP using deterministic tools and the job_filter_agent.


IMPORTANT
---------

The number of jobs you need to find is:
<profile.job_preferences.number_of_jobs_wanted>

==============================================================
TOOLS YOU MUST USE
==============================================================

1. job_finder_tool
   RETURNS:
   <JOB_FINDER_OUTPUT_SCHEMA>

2. fetch_job_tool
   RETURNS:
   {
      "url": <string>,
      "status": <int>,
      "success": <bool>,
      "content": <string>,
      "error": <string or null>
   }

3. extract_job_details_agent  (sub agent)
   RETURNS:
   {
      "job_id": <string>,
      "title": <string>,
      "company": <string>,
      "location": <string>,
      "employment_type": <string>,
      "salary": <string or null>,
      "job_description": <string>,
      "requirements": [<string>],
      "qualifications": [<string>],
      "skills_mentioned": [<string>],
      "apply_url": <string>
   }

4. normalize_job_details_tool  (deterministic)
   PURPOSE:
   Complete and normalize job details to satisfy JOB_DETAILS_SCHEMA.
   - Adds job_id (sha256(url))
   - Adds apply_url
   - Fills missing fields required by schema
   RETURNS:
   <JOB_DETAILS_SCHEMA>


5. normalize_job_details_tool
   PURPOSE:
   Deterministically complete and normalize the job details.
   - Adds job_id (sha256(url))
   - Adds apply_url = url
   - Fills all missing fields required by JOB_DETAILS_SCHEMA
   - Ensures schema completeness

   RETURNS:
   <JOB_DETAILS_SCHEMA>

5. job_filter_agent  (sub-agent)
   RETURNS:
   {
      "pass": <bool>,
      "score": <0-100>,
      "rationale": <string>
   }

6. rank_job_tool_adk
   RETURNS:
   {
       "ranked_jobs": [<job objects>],
       "top_k": <int>,
       "total_jobs_in": <int>,
       "total_jobs_ranked": <int>
   }


==============================================================
LOOP BEHAVIOR (EXTREMELY IMPORTANT)
==============================================================


Before starting the loop, you MUST define these internal variables:

• collected_jobs = []        # a list that stores ONLY the jobs that pass filtering
• number_of_passed_jobs = 0  # counter for how many passed jobs you have found
• target_jobs = <profile.job_preferences.number_of_jobs_wanted> # how many jobs you need to find



1. Use job_finder_tool_adk to get job_links.

2. For EACH job_link:
       - Call fetch_job_tool_adk(url)
       - If fetch failed → skip
       - Call extract_job_details_agent(url, raw_content)
        - Call normalize_job_details_tool with:
              {
                 "url": url,
                 "agent_output": <output from extract_job_details_agent>
              }
        
        - Call job_filter_agent with the normalized job details:
              {
                  "job_details": <output from normalization tool>,
                  "profile": <PROFILE_SCHEMA>,
                  "rejection_memory": rejection_memory
              }

       - Attach:
            job["score"] = filter.score
            job["pass"]  = filter.pass
            job["rationale"] = filter.rationale

       - KEEP ONLY passed jobs

3. Continue looping UNTIL:
    number_of_passed_jobs >= target_jobs

4. When enough jobs collected:
       Call rank_job_tool_adk with:
       {
           "jobs": collected_jobs,
           "top_k": target_jobs
       }

       The tool returns a ranked list.

==============================================================
FINAL OUTPUT SCHEMA TO ORCHESTRATOR
==============================================================
{
    "ranked_jobs": [ <job dicts after ranking> ]
}

This MUST be the only output.

==============================================================
RULES
==============================================================

- NEVER hallucinate job data.
- NEVER generate job text yourself — always use extract tool.
- NEVER override scores manually — job_filter_agent controls scoring.
- NEVER contact user.
- You MUST stop when you reach the requested number of passing jobs.
- Use SESSION memory only for temporary result storage.
- Output MUST follow the final schema exactly.
"""
)

print("✅ Job Search Agent (Agent 1) rewritten with full schema precision.")


✅ Extract Job Details Agent updated (semantic only, ADK-correct).
✅ Job Search Agent (Agent 1) rewritten with full schema precision.


In [29]:
# ============================================================
# CELL — Job Summarizer Agent (Corrected Version)
# ============================================================

from google.adk.agents import LlmAgent

job_summarizer_agent = LlmAgent(
    model=gemini_lite,
    name="job_summarizer_agent",
    description=(
        "Generates clear, concise summaries of job postings for the user. "
        "Takes structured job JSON and returns a short, readable summary."
    ),
    static_instruction="""
You are the **Job Summarizer Agent** in JobPilot.

Your job:
Given a structured job JSON produced by the extraction tools, summarize the job
in a short, clear, user-friendly way.

INPUT:
A single job JSON object containing fields such as:
    • job_id
    • title
    • company
    • location
    • employment_type
    • salary (if available)
    • job_description
    • requirements / qualifications
    • skills_mentioned
    • apply_url
    • score (0–100) from job_filter_agent

OUTPUT (strict schema):
{
    "job_id": "<job_id>",
    "summary": "<2–5 sentence readable summary>",
    "score": <0–100>,
    "link": "<url>"   <-- This must come from input.apply_url
}

GUIDELINES:
- Produce a **human-friendly 2–5 sentence** summary.
- Include:
      • what the company is looking for
      • why it might be a good fit (based on the score context)
      • notable requirements or highlights
- NEVER invent details that are not in the job JSON.
- ALWAYS use job.apply_url as the link field in the output.
- NEVER modify job fields except rewriting them in natural language.
- Do NOT:
      • decide whether the user should apply
      • filter jobs
      • call tools
      • store memory

You ONLY transform the structured job data into a readable summary.
""",
)

print("✅ Job Summarizer Agent created and schema-consistent with all tools.")


✅ Job Summarizer Agent created and schema-consistent with all tools.


In [30]:
# ============================================================
# CELL — Application Builder Agent + Resume & Cover Letter Sub-Agents
# ============================================================

from google.adk.agents import LlmAgent
import json


# ------------------------------------------------------------
# RESUME GENERATOR SUB-AGENT (High-Accuracy Version)
# ------------------------------------------------------------
resume_generator_agent = LlmAgent(
    model=gemini_flash,
    name="resume_generator_agent",
    description=(
        "Generates a fully tailored resume for a specific job, using the "
        "user's structured profile and the job's structured details."
    ),
    static_instruction="""
You are the **Resume Generator Agent** in JobPilot.

Your task:
Given:
    • user_profile: JSON strictly following PROFILE_SCHEMA
    • job: JSON strictly following JOB_DETAILS_SCHEMA (with added fields: score, pass, rationale)
Produce a **professionally written, tailored resume** for that job.

==============================================================
INPUT FORMAT
==============================================================
{
    "user_profile": {
        "name": <string>,
        "location": <string>,
        "contact": {...},
        "education": [...],
        "experience": [...],
        "skills": [...],
        "job_preferences": {...},
        "additional_notes": <string>,
        "update_required": <bool>,
        "last_update": <timestamp>
    },
    "job": {
        "job_id": <string>,
        "title": <string>,
        "company": <string>,
        "location": <string>,
        "employment_type": <string>,
        "salary": <string or null>,
        "job_description": <string>,
        "requirements": [<string>],
        "qualifications": [<string>],
        "skills_mentioned": [<string>],
        "apply_url": <string>,
        "score": <0–100>,
        "pass": <bool>,
        "rationale": <string>
    }
}

==============================================================
OUTPUT SCHEMA (STRICT)
==============================================================
{
    "job_id": "<same job.job_id>",
    "resume_text": "<professionally formatted tailored resume>"
}

==============================================================
RESUME RULES
==============================================================

- MUST be tailored to the specific job's:
      • responsibilities
      • requirements
      • preferred skills
- MUST prioritize relevant parts of the user_profile.
- MUST NOT fabricate:
      • degrees
      • job titles
      • companies
      • certifications
      • skills the user does not list
- You MAY:
      • restructure experience
      • rewrite bullet points for clarity
      • emphasize matching skills or achievements
- Tone must be polished, professional, and concise.
- Layout can be bullet points or paragraphs.
- Output MUST be strictly JSON, no extra text.

==============================================================
END OF SPECIFICATION
==============================================================
"""
)


# ------------------------------------------------------------
# COVER LETTER GENERATOR SUB-AGENT (High-Accuracy Version)
# ------------------------------------------------------------
cover_letter_agent = LlmAgent(
    model=gemini_flash,
    name="cover_letter_generator_agent",
    description=(
        "Generates a tailored cover letter for a job based on the user's "
        "experience, skills, and motivations."
    ),
    instruction="""
You are the **Cover Letter Generator Agent** in JobPilot.

Your task:
Given:
    • user_profile (PROFILE_SCHEMA)
    • job (JOB_DETAILS_SCHEMA + score + rationale)
produce a **tailored 2–4 paragraph cover letter**.

==============================================================
INPUT FORMAT
==============================================================
{
    "user_profile": { ... },
    "job": { ... }
}

==============================================================
OUTPUT SCHEMA (STRICT)
==============================================================
{
    "job_id": "<string>",
    "cover_letter_text": "<string>"
}

==============================================================
COVER LETTER RULES
==============================================================

CONTENT:
- Explain:
      • Why the user is a strong match
      • Relevant experience & skills tied directly to the job requirements
      • Tangible value the user offers the company
      • Motivation for the role and/or company

TONE:
- Professional, warm, confident.
- NOT generic; MUST reference job.title and job.company.
- Use user's profile information.
- NEVER fabricate degrees, companies, or skills.

STRUCTURE:
- 2–4 paragraphs
- Must be coherent and personalized.

CONSTRAINTS:
- NO tool calls.
- NO user interaction.
- ONLY output valid JSON.
- No markdown.

==============================================================
END OF SPECIFICATION
==============================================================
"""
)


# ------------------------------------------------------------
# APPLICATION BUILDER AGENT (Agent 2 — High-Accuracy Version)
# ------------------------------------------------------------
application_builder_agent = LlmAgent(
    model=gemini_flash,
    name="application_builder_agent",
    description=(
        "Agent 2 in JobPilot. Coordinates resume and cover letter generation "
        "for the user's selected jobs."
    ),
    instruction="""
You are the **Application Builder Agent** (Agent 2) in JobPilot.

Your job:
Take the final selected job list from the orchestrator and produce complete application packages.

==============================================================
EXPECTED INPUT SCHEMA
==============================================================
{
    "selected_jobs": [
        <job JSON objects from Agent 1, each containing: 
          job_id, title, company, location,
          employment_type, salary,
          job_description, requirements, qualifications,
          skills_mentioned, apply_url,
          score, pass, rationale>
    ],
    "user_profile": <PROFILE_SCHEMA>
}

==============================================================
PROCESS
==============================================================

For EACH job in selected_jobs:

1. Call resume_generator_agent with:
   {
       "user_profile": <profile>,
       "job": <job JSON>
   }

2. Call cover_letter_generator_agent with:
   {
       "user_profile": <profile>,
       "job": <job JSON>
   }

Collect both outputs.

==============================================================
OUTPUT SCHEMA (STRICT)
==============================================================
{
    "applications": [
        {
            "job_id": "<string>",
            "resume_text": "<string>",
            "cover_letter_text": "<string>"
        },
        ...
    ]
}

==============================================================
RULES
==============================================================

- NEVER generate resume_text or cover_letter_text yourself — always call the sub-agents.
- NEVER modify job data or profile data.
- MUST return JSON ONLY.
- MUST keep internal reasoning minimal.
- MUST return applications in the SAME order as selected_jobs.
- No contacting the user — orchestrator handles communication.
- If a sub-agent returns invalid JSON, you must fail gracefully with a structured error.

==============================================================
END OF SPECIFICATION
==============================================================
"""
)

print("✅ Application Builder Agent + Sub-Agents rewritten with full accuracy and consistency.")


✅ Application Builder Agent + Sub-Agents rewritten with full accuracy and consistency.


In [51]:
# ============================================================
# CELL — Attach tools to agents (correct ADK syntax)
# ============================================================

from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.function_tool import FunctionTool

# ------------------------------------------------------------
# Create ADK wrappers for LLM sub-agents (required!)
# ------------------------------------------------------------
extract_job_details_tool_adk = AgentTool(agent=extract_job_details_agent)
profile_builder_agent_adk = AgentTool(agent=profile_builder_agent)
job_filter_agent_adk = AgentTool(agent=job_filter_agent)
resume_generator_agent_adk = AgentTool(agent=resume_generator_agent)
cover_letter_agent_adk = AgentTool(agent=cover_letter_agent)

# ------------------------------------------------------------
# Orchestrator tools (entry point)
# ------------------------------------------------------------
orchestrator_agent.tools = [
    # Profile Builder AGENT (replaces old deterministic tool)
    profile_builder_agent_adk,

    # Job Search Agent
    AgentTool(agent=job_search_agent),

    # Job Summarizer Agent
    AgentTool(agent=job_summarizer_agent),

    # Application Builder Agent
    AgentTool(agent=application_builder_agent),
]

# ------------------------------------------------------------
# Job Search Agent tools
# (Order matches the job-search loop)
# ------------------------------------------------------------
job_search_agent.tools = [
    job_finder_tool_adk,
    fetch_job_tool_adk,

    # Extract job details (LLM)
    extract_job_details_tool_adk,

    # Deterministic normalization (fills job_id, apply_url, missing fields)
    normalize_job_details_tool,

    # Rank jobs
    rank_job_tool_adk,

    # Filter agent (sub-agent)
    job_filter_agent_adk,
]

# ------------------------------------------------------------
# Job Filter Agent (pure reasoning)
# ------------------------------------------------------------
job_filter_agent.tools = []  

# ------------------------------------------------------------
# Job Summarizer Agent (pure reasoning)
# ------------------------------------------------------------
job_summarizer_agent.tools = []

# ------------------------------------------------------------
# Resume & Cover Letter Generators (pure reasoning)
# ------------------------------------------------------------
resume_generator_agent.tools = []
cover_letter_agent.tools = []

# ------------------------------------------------------------
# Application Builder Agent tools
# (calls resume + cover letter sub-agents)
# ------------------------------------------------------------
application_builder_agent.tools = [
    resume_generator_agent_adk,
    cover_letter_agent_adk,
]

print("✅ ALL AGENT + TOOL WIRING UPDATED, CLEAN, AND ADK-CORRECT")


✅ ALL AGENT + TOOL WIRING UPDATED, CLEAN, AND ADK-CORRECT


In [44]:
# ============================================================
# Runners + LoggingPlugin
# ============================================================

# Import the base Runner class, which handles persistence
from google.adk.runners import Runner
from google.adk.plugins.logging_plugin import LoggingPlugin

# --- Variable Definition (already fixed) ---
APP_NAME = "JobPilot_AgentSystem" 
# ----------------------------------------

# Instantiate the persistent runner
runner = Runner( # <-- Changed from InMemoryRunner to Runner
    agent=orchestrator_agent,
    app_name=APP_NAME,
    session_service=session_service, # <-- This argument is now accepted
    plugins=[LoggingPlugin()],
)

print("✅ Persistent Runner (Runner) + Logging Plugin configured.")

✅ Persistent Runner (Runner) + Logging Plugin configured.


In [47]:
# ============================================================
# Basic sanity test
# ============================================================

print("🚀 Starting FULL JobPilot Test Run...\n")

test_input = """
Hi, my name is Ofer Harpaz.

I’m currently based in Rockville, MD.  
My phone number is 240-316-0830 and my email is oferharvai@gmail.com.  
My LinkedIn is linkedin.com/in/ofer-v-data-analysis.

About me:  
I’m a data analyst and machine learning practitioner with experience in Python, AI agents, data pipelines, and research-driven model design.  
I enjoy solving problems, building structured AI systems, and doing high-frequency, high-intensity training.  
  
Education:  
- BSc in Mathematics (with honors) from the Open University of Israel (2024).  
- Additional coursework: linear algebra, probability, statistics, optimization, machine learning, deep learning.

Experience:  
- Tutoring math, statistics, and programming for several years.  
- Built multiple agentic AI systems including JobPilot and a research agent for academic paper retrieval.  
- Worked on analytics projects involving data cleaning, feature engineering, ensemble models, and LLM-driven automation.  
- Experience coaching kids at a climbing gym.

Technical Skills:  
Python, data analysis, statistics, NumPy, Pandas, scikit-learn, TensorFlow, PyTorch, machine learning, deep learning, CNNs, RNNs, Transformers, optimization, AI agents, scraping, SQL, API work, orchestration, prompt engineering, session management, MLOps basics.

Soft Skills:  
Communication, teaching, problem-solving, high adaptability, structured thinking, instructional design.

Certifications:  
- NASM CPT  
- CPR/AED  
- Additional: some ML courses, LLM courses, Kaggle Agents Intensive.

Projects:  
- JobPilot (multi-agent job application automation system)  
- Antisemitism detection model using multilayer GPT reasoning  
- Research assistant agent for literature review  
- Fake news classification project  
- Machine learning models including bagging, boosting, XGBoost, KNN, SVMs, CNNs, RNNs, Transformers.

Job Preferences:  
I’m looking for roles such as:  
- Data Analyst  
- Machine Learning Engineer  
- AI Engineer  
- Applied AI Engineer  
- Research Engineer (entry-level)

Work Mode Preferences:  
I prefer remote or hybrid roles.

Location Preferences:  
Anywhere in the US is fine, but ideally EST time zone.

Industry Preferences:  
Tech, education, AI, analytics, or research organizations.

Salary Expectations:  
Ideally around 80–110k, but flexible depending on role.

Other Preferences:  
- I want around 3 job suggestions at a time.  
- I prefer smaller teams or fast-moving startups.  
- I enjoy roles with learning opportunity and growth.  

Let me know what roles you find.

"""

response = await runner.run_debug(test_input)

print("\n============================")
print("🟢 Test Run Complete")
print("============================")

response


🚀 Starting FULL JobPilot Test Run...


 ### Continue session: debug_session_id

User > 
Hi, my name is Ofer Harpaz.

I’m currently based in Rockville, MD.  
My phone number is 240-316-0830 and my email is oferharvai@gmail.com.  
My LinkedIn is linkedin.com/in/ofer-v-data-analysis.

About me:  
I’m a data analyst and machine learning practitioner with experience in Python, AI agents, data pipelines, and research-driven model design.  
I enjoy solving problems, building structured AI systems, and doing high-frequency, high-intensity training.  
  
Education:  
- BSc in Mathematics (with honors) from the Open University of Israel (2024).  
- Additional coursework: linear algebra, probability, statistics, optimization, machine learning, deep learning.

Experience:  
- Tutoring math, statistics, and programming for several years.  
- Built multiple agentic AI systems including JobPilot and a research agent for academic paper retrieval.  
- Worked on analytics projects involving data cleanin

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Content: function_call: profile_builder_agent
[logging_plugin]    Token Usage - Input: 16035, Output: 700
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 2d969011-0347-4dc7-97de-92b6000ae591
[logging_plugin]    Author: orchestrator_agent
[logging_plugin]    Content: function_call: profile_builder_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['profile_builder_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: profile_builder_agent
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Function Call ID: adk-1f59ccd8-9379-4916-87a6-18c3d002772d
[logging_plugin]    Arguments: {'request': 'Hi, my name is Ofer Harpaz.\n\nI’m currently based in Rockville, MD.  \nMy phone number is 240-316-0830 and my email is oferharvai@gmail.com.  \nMy LinkedIn is linkedin.com/in/ofer-v-data-analysis.\n\nAbout me:  \nI’m a data

[Event(model_version='gemini-2.5-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'request': """Hi, my name is Ofer Harpaz.
 
 I’m currently based in Rockville, MD.  
 My phone number is 240-316-0830 and my email is oferharvai@gmail.com.  
 My LinkedIn is linkedin.com/in/ofer-v-data-analysis.
 
 About me:  
 I’m a data analyst and machine learning practitioner with experience in Python, AI agents, data pipelines, and research-driven model design.  
 I enjoy solving problems, building structured AI systems, and doing high-frequency, high-intensity training.  
   
 Education:  
 - BSc in Mathematics (with honors) from the Open University of Israel (2024).  
 - Additional coursework: linear algebra, probability, statistics, optimization, machine learning, deep learning.
 
 Experience:  
 - Tutoring math, statistics, and programming for several years.  
 - Built multiple agentic AI systems including JobPilot and a research agent f